# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

In [1]:
# Our goal is to utilize historical data on car sales and associated car features (year, color, make, etc.) in order to identify underlying relationship between features and the resulting sale price. Car price in this case is the dependent variable (y) while the set of feature characteristics of specific cars is a matrix of independent variables (X). The modelled relationship will be used to forecast potential sale price of new cars based on known combination of their features.

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [62]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.inspection import permutation_importance
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline

In [63]:
# load up the dataset and do basic cleaning data exploration
df = pd.read_csv('data/vehicles.csv')

In [64]:
# explore overall variety of existing features, their data types & whether any nulls are present
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

In [65]:
# Overall, some columns have only few NaN values which we'd be fine dropping, however, others (such as 'size', 'type' or 'cyllinders') have quite a few, dropping them would leave us with less than 50% of the initial sample. Let's explore the dataset further to see potential relationships between NaNs across columns.

In [66]:
# With size being a column with most NaNs, let's explore whether some types just don't have an associated size in the data
df.query('~size.isnull() and ~type.isnull()')['type'].unique()

array(['truck', 'SUV', 'pickup', 'mini-van', 'sedan', 'offroad', 'bus',
       'van', 'convertible', 'coupe', 'hatchback', 'wagon', 'other'],
      dtype=object)

In [67]:
df.query('size.isnull() and ~type.isnull()')['type'].unique()

array(['pickup', 'truck', 'other', 'coupe', 'SUV', 'hatchback', 'sedan',
       'convertible', 'wagon', 'van', 'bus', 'mini-van', 'offroad'],
      dtype=object)

In [68]:
# It looks like some types indeed do not have an assigned size, however some entries simply have a missing value.

In [69]:
# Let's also look at the overall statistics of numerical variables to make sure they are 'alright'
df.describe()

,id,price,year,odometer
count,4.268800e+05,4.268800e+05,425675.000000,4.224800e+05
mean,7.311487e+09,7.519903e+04,2011.235191,9.804333e+04
std,4.473170e+06,1.218228e+07,9.452120,2.138815e+05
min,7.207408e+09,0.000000e+00,1900.000000,0.000000e+00
25%,7.308143e+09,5.900000e+03,2008.000000,3.770400e+04
50%,7.312621e+09,1.395000e+04,2013.000000,8.554800e+04
75%,7.315254e+09,2.648575e+04,2017.000000,1.335425e+05
max,7.317101e+09,3.736929e+09,2022.000000,1.000000e+07


In [70]:
# It seems like there are some outliers and null values in the price and odometer columns likely a result of manual mistyping
df.sort_values('odometer', ascending = False).head(5)

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
262338,7313707752,las cruces,4800,1949.0,ford,f1,good,8 cylinders,gas,10000000.0,missing,manual,NaN,rwd,full-size,pickup,blue,nm
292399,7315639901,cleveland,1300,1950.0,chevrolet,NaN,fair,8 cylinders,gas,10000000.0,clean,automatic,NaN,rwd,full-size,sedan,red,oh
108797,7313810434,sarasota-bradenton,450,2000.0,ford,f-150,NaN,NaN,gas,10000000.0,clean,automatic,NaN,NaN,NaN,NaN,NaN,fl
346469,7313602958,florence,1700,1973.0,NaN,1973 nova,NaN,NaN,other,10000000.0,clean,other,NaN,NaN,NaN,NaN,NaN,sc
390252,7304207596,danville,2000,1964.0,ford,f-150,fair,6 cylinders,gas,10000000.0,clean,other,NaN,rwd,NaN,NaN,NaN,va


### Data Preparation

After our initial exploration and fine tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

In [71]:
# Based on the initial analysis of data quality we'll need to do several steps to 'clean' the data: 
# - for price and odometer columns we'll remove extreme values / outliers defined by +/- 3 standard deviations from the mean of each respective column
# - we'll also remove null values for other categorical variables except size drive and paint color
# - we'll also remove VIN column altogether as it is just a unique vehicle identifier.

In [72]:
columns_to_drop = ['price','year','manufacturer','model','condition','cylinders','fuel','odometer','title_status','transmission','drive', 'type', 'paint_color', 'state']

In [73]:
df = df.dropna(subset = columns_to_drop)
df = df.drop(['id','VIN'], axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115988 entries, 31 to 426878
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        115988 non-null  object 
 1   price         115988 non-null  int64  
 2   year          115988 non-null  float64
 3   manufacturer  115988 non-null  object 
 4   model         115988 non-null  object 
 5   condition     115988 non-null  object 
 6   cylinders     115988 non-null  object 
 7   fuel          115988 non-null  object 
 8   odometer      115988 non-null  float64
 9   title_status  115988 non-null  object 
 10  transmission  115988 non-null  object 
 11  drive         115988 non-null  object 
 12  size          79195 non-null   object 
 13  type          115988 non-null  object 
 14  paint_color   115988 non-null  object 
 15  state         115988 non-null  object 
dtypes: float64(2), int64(1), object(13)
memory usage: 15.0+ MB


In [74]:
# We are left with ~ a third of a dataset with most values present, let's replace 'size' NaN with 'unknown' value 
df.fillna('unknown', inplace = True)

In [75]:
# Let's remove outliers based on price and odometer columns based on the +/- 2 std rule and 0 for price
price_upper = df['price'].median() + 2*df['price'].std()
price_lower = df['price'].median() - 2*df['price'].std()
odometer_upper = df['odometer'].median() + 2*df['odometer'].std()
odometer_lower = df['odometer'].median() - 2*df['odometer'].std()
df = df.query('price > @price_lower and price < @price_upper and price < 1000000 and odometer > @odometer_lower and odometer < @odometer_upper and price > 1')

In [76]:
# Let's also replace column year with age counted from current year 
df['age'] = 2022 - df['year']

In [77]:
df = df.drop('year', axis = 1)

In [78]:
# Let's split the data into sets of independent variables X and dependent variable y for further modelling & split into train and test sets for final validation. Additionally, we'll use natural log of y as the target variable to avoid negative predictions
X = df.drop('price', axis = 1)
y = np.log(df['price'])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 32)

In [79]:
df['price'].sort_values(ascending = True)

380551         3
297060         3
102494         4
102428         4
261436         4
           ...  
123206    155000
323687    155000
113503    165000
104511    169999
201823    195000
Name: price, Length: 110526, dtype: int64

In [80]:
# Let's build a pipeline to transform categorical variables into dummies & polynomial transformation for odometer and age
numeric_transformer = Pipeline(steps=[('poly', PolynomialFeatures(include_bias=False)),('scaler', StandardScaler())])

In [81]:
numeric_features = list(X_train.select_dtypes('float64').columns)
categorical_features = list(X_train.select_dtypes('object').columns)

In [82]:
categorical_transformer = Pipeline([('onehot', OneHotEncoder(handle_unknown = 'ignore'))])

In [83]:
full_transformer = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),('cat', categorical_transformer, categorical_features)])

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

In [84]:
ridge_pipe = Pipeline([('transform', full_transformer ),('linreg', Ridge())])

In [85]:
param_dict = {'linreg__alpha' : [0.1,1,10,100], 'transform__num__poly__degree' : [1,2,3]}

In [86]:
grid = GridSearchCV(estimator = ridge_pipe, param_grid = param_dict)

In [87]:
grid.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('transform',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('poly',
                                                                                          PolynomialFeatures(include_bias=False)),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['odometer',
                                                                          'age']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('onehot',
                                                                              

In [88]:
grid.best_params_

{'linreg__alpha': 1, 'transform__num__poly__degree': 3}

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight on drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

In [89]:
# Now that the model has been fit, let's assess mean absolute error based on test and train data

In [90]:
y_hat_test = grid.best_estimator_.predict(X_test)
test_mae = mean_absolute_error(np.exp(y_test), np.exp(y_hat_test))
mean_y_test = np.exp(y_test).mean()
print(test_mae/mean_y_test)

0.19990908328158755


In [91]:
# The estimated model is able to accurately predict sale price within 20% accuracy.

In [92]:
# Let's also explore what features influence the sale price the most

In [93]:
r = permutation_importance(grid.best_estimator_, X_test, y_test)

In [94]:
importances = pd.concat([pd.DataFrame(X_test.columns), pd.DataFrame(r['importances_mean'])], axis = 1)
importances.columns = ['variable', 'importance_mean']

In [95]:
fig = px.bar(importances.sort_values('importance_mean',ascending=True), x = 'importance_mean', y = 'variable', title = 'Factors with most influence on price (mean importance)')

In [96]:
# Let's also evaluate coefficients and their impact on the sale price using transformation parameters from our best gridsearch instance
n =  PolynomialFeatures(degree=3,include_bias=False)
n_fit = n.fit(X_test[numeric_features])
numeric_names = n_fit.get_feature_names(numeric_features)
c = OneHotEncoder()
c_fit = c.fit(X_train[categorical_features])
cat_names = c_fit.get_feature_names(categorical_features)

In [97]:
all_variables = list(numeric_names) + list(cat_names)

In [98]:
# build dataframe with coefficients of the best estimator
coefficients = pd.concat([pd.DataFrame(all_variables),pd.DataFrame(grid.best_estimator_.named_steps['linreg'].coef_)], axis = 1)
coefficients.columns = ['feature', 'coefficient']

In [99]:
# Explore top coefficients by various cuts
coefficients.sort_values('coefficient', ascending=True).head(20)

,feature,coefficient
1231,model_4runner sr5 sport awd,-4.790400
4541,model_f-250 super duty ext-cab,-4.336923
7786,model_ridgeline 4wd,-3.834046
8900,model_srx 4,-3.084352
4558,model_f-250 xlt lariat,-2.972042
8459,model_sierra xlt crew,-2.931027
5635,model_grand cherokee altitude,-2.853820
8482,model_silverado 1500 crew c,-2.837434
7838,model_roadster 2000,-2.769922
2938,model_civic turbo ex-l,-2.759868


In [100]:
coefficients.sort_values('coefficient', ascending = True).tail(20)

,feature,coefficient
3057,model_comanche,1.148398
9887,model_viper rt/10,1.151846
2563,model_camper,1.158938
9792,model_vanagon,1.168585
2533,model_camaro iroc-z28,1.182923
2203,model_benz g500,1.199632
3581,model_delica reimo,1.232367
9936,model_westfalia,1.261360
10244,model_xk150 se dhc,1.274597
9795,model_vanagon syncro,1.302322


In [101]:
coefficients.query('~feature.str.contains("model")').sort_values('coefficient', ascending = False).head(15)

,feature,coefficient
4,age^2,1.597445
421,manufacturer_ferrari,1.377043
442,manufacturer_porsche,0.849150
447,manufacturer_tesla,0.816139
412,manufacturer_aston-martin,0.743592
10417,cylinders_12 cylinders,0.503746
10424,fuel_diesel,0.366037
10410,condition_excellent,0.345008
10430,title_status_lien,0.299567
10413,condition_like new,0.294669


In [102]:
coefficients.query('~feature.str.contains("model")').sort_values('coefficient', ascending = True).head(15)

,feature,coefficient
373,region_tuscarawas co,-1.325297
1,age,-1.145102
10447,type_bus,-0.709734
8,age^3,-0.689665
42,region_boise,-0.577281
10415,condition_salvage,-0.572060
409,region_zanesville / cambridge,-0.544978
10432,title_status_parts only,-0.542574
49,region_brownsville,-0.478515
189,region_lakeland,-0.464981


In [103]:
coefficients.query('feature.str.contains("condition")').sort_values('coefficient', ascending = True)

,feature,coefficient
10415,condition_salvage,-0.572060
10411,condition_fair,-0.388431
10414,condition_new,0.104782
10412,condition_good,0.216033
10413,condition_like new,0.294669
10410,condition_excellent,0.345008


In [104]:
coefficients.query('feature.str.contains("paint")').sort_values('coefficient', ascending = True)

,feature,coefficient
10461,paint_color_brown,-0.055296
10463,paint_color_green,-0.037451
10466,paint_color_purple,-0.030363
10467,paint_color_red,-0.024859
10468,paint_color_silver,-0.015553
10460,paint_color_blue,0.001442
10469,paint_color_white,0.001865
10459,paint_color_black,0.009516
10464,paint_color_grey,0.012383
10465,paint_color_orange,0.042856


In [105]:
coefficients.query('feature.str.contains("title")').sort_values('coefficient', ascending = True)

,feature,coefficient
10432,title_status_parts only,-0.542574
10431,title_status_missing,-0.354765
10434,title_status_salvage,0.127023
10433,title_status_rebuilt,0.185920
10429,title_status_clean,0.284829
10430,title_status_lien,0.299567


In [111]:
coefficients.query('feature.str.contains("age") and ~feature.str.contains("model")').sort_values('coefficient', ascending = True)

,feature,coefficient
1,age,-1.145102
8,age^3,-0.689665
10415,condition_salvage,-0.572060
3,odometer age,-0.321456
449,manufacturer_volkswagen,-0.179875
6,odometer^2 age,0.091657
7,odometer age^2,0.123861
10434,title_status_salvage,0.127023
17,region_anchorage / mat-su,0.150759
4,age^2,1.597445


In [106]:
coefficients.query('feature.str.contains("manufacturer")').sort_values('coefficient', ascending = True)

,feature,coefficient
445,manufacturer_saturn,-0.437797
431,manufacturer_kia,-0.403146
437,manufacturer_mercury,-0.379939
422,manufacturer_fiat,-0.368270
418,manufacturer_chrysler,-0.320341
427,manufacturer_hyundai,-0.306984
435,manufacturer_mazda,-0.282697
439,manufacturer_mitsubishi,-0.247203
440,manufacturer_nissan,-0.232290
419,manufacturer_datsun,-0.225195


### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine tuning their inventory.

        After careful analysis of provided data on more than 400k sales of used vehicles, the team was able to build a linear regression model predicting sale price based on a number of characteristics of each vehicle (model, condition, age, etc.) with average accuracy of +/- 20%.  

        In addition to leveraging the model to predict a recommended listing price for a new vehicle, the model's greater value is its ability to identify price impact of specific characteristics on the final price which you will be able to use when making assessments of potential vehicles.

    Based on the number of reviewed characteristics, age of the car has the most significant impact on the overall value. Unfortunately, the relationship is not linear so there is no simple rule subtracting a specific amount for each year. Generally, older vehicles have a more significant negative impact on price than newer.

In [107]:
fig.show()

        As expected, specific models also have a huge impact on price that varies wildly across models (both positive and negative). Some examples include: 
        - Skyline gtr 32 and Eurovan Camper increase price by almost 180%
        - 4runner sport AWD and F-250 super duty lead to almost a x4 price decrease
    Note: Specific model impact for cars with few sales may be volatile, use caution. Full model list and their impact is provided in appendix.

        Manufacturer of the car is another important characteristic impacting car price. Some of the top and bottom makes and their impact are:
    - Saturn, Kia, Mercury and Fiat are almost 40% cheaper
    - Ferrari is the most expensive with almost 180% higher price
    - Porsche, Tesla and Astom-Martin have 85%, 82% and 74% higher price respectively

    Condition also has a signficant impact on price with Excellent and Like New commanding a 34% and 29% price premium respecitvely, while Salvage and Fair deprecating sale price by 57% and 37% respectively

        Finally, while paint color has a much smaller impact than other variables, some colors do perform better than others with regards to the sale price:
        - Yellow, custom and orange have a 5% and 4% premium respecitvely
        - Brown, green and purple decrease price by 5.5%, 3.7% and 3% respecitvely

        Finally, in addition to using the above characteristics when making pricing decisions for upcoming car listings, we recommend:
        - Where possible, taking advantage from potential arbitrage when procuring used vehicles (i.e. if two different colors have the same price but one color has higher impact on final sale, get that color)
        - For further improvements of the model and understanding, create improvements to the process of data collection - as part of the analysis we found a lot of mistyped data entires or simply missing information - out of the initial dataset of over 400k items, we were able to use less than a third for modelling. While that didn't impact the ability to create a model, more data equals better precision and will help with future revisions to the model